In [1]:
!wandb login "TOKEN"

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [2]:
!git clone https://github.com/howard-hou/RWKV-X

Cloning into 'RWKV-X'...
remote: Enumerating objects: 10203, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 10203 (delta 97), reused 108 (delta 55), pack-reused 10046 (from 2)
Receiving objects: 100% (10203/10203), 17.97 MiB | 7.05 MiB/s, done.
Resolving deltas: 100% (5479/5479), done.


In [3]:
# Install the CUDA 12.1 version of torch by specifying the URL
!pip install torch --upgrade
# Specify to install the 1.9.5 version of PyTorch Lightning and other commonly used machine learning toolkits
# The --upgrade parameter means that if the corresponding software package is already installed in the environment, it will be upgraded to the latest version
!pip install pytorch-lightning==1.9.5 deepspeed wandb ninja --upgrade
# The following are some other toolkits that will be used when training with RWKV-PEFT. It is recommended to install them in advance
!pip install bitsandbytes einops triton rwkv-fla rwkv transformers GPUtil plotly gradio datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 158.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch # Import the torch package

# Print the PyTorch version
print(f"PyTorch version: {torch.__version__}")
# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
# Print the CUDA version
print(f"CUDA version: {torch.version.cuda}")
# Print the number of available GPUs
print(f"Number of GPUs: {torch.cuda.device_count()}")


PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8
Number of GPUs: 1


In [5]:
!pip install pyrwkv-tokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 96.8 MB/s eta 0:00:00


In [6]:
!pip uninstall torch torchvision torchaudio

Found existing installation: torch 2.9.1
Uninstalling torch-2.9.1:
  Would remove:
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.9.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.9.1
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/torchvision-0.21.0+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libcudart.41118559.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libjpeg.1c1c4b09.so.8
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libnvjpeg.02b6d700.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libpng16.0364a1db.so.16
    /usr/local/lib/python3.11/dist-packa

In [7]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 130.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 153.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [14]:
%cd /content/RWKV-X/pretrain/

/content/RWKV-X/pretrain


### Предварительно необходимо загрузить [датасет](https://huggingface.co/datasets/LucyRez/RedPajama-4k-dataset/) в папку pretrain

In [15]:
!python make_data.py RedPajama-15B-4k-dataset.jsonl 5 4096

### Convert RedPajama-15B-4k-dataset.jsonl to /content/RWKV-X/pretrain/RedPajama-15B-4k-dataset_binidx/RedPajama-15B-4k-dataset.bin/idx...
### Shuffle: 1 out of 5
### Found 97657 non-empty lines in RedPajama-15B-4k-dataset.jsonl
### Split data into 100 piles.
### Temporary files are stored in 'temp_piles' directory.
### Shuffle: 2 out of 5
### Found 97657 non-empty lines in RedPajama-15B-4k-dataset.jsonl
### Split data into 100 piles.
### Temporary files are stored in 'temp_piles' directory.
### Shuffle: 3 out of 5
### Found 97657 non-empty lines in RedPajama-15B-4k-dataset.jsonl
### Split data into 100 piles.
### Temporary files are stored in 'temp_piles' directory.
### Shuffle: 4 out of 5
### Found 97657 non-empty lines in RedPajama-15B-4k-dataset.jsonl
### Split data into 100 piles.
### Temporary files are stored in 'temp_piles' directory.
### Shuffle: 5 out of 5
### Found 97657 non-empty lines in RedPajama-15B-4k-dataset.jsonl
### Split data into 100 piles.
### Temporary files are 

In [9]:
!pip uninstall flash-attn
!pip install flash-attn==2.7.4.post1 --no-build-isolation

Found existing installation: flash_attn 2.7.4.post1
Uninstalling flash_attn-2.7.4.post1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/flash_attn-2.7.4.post1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/flash_attn/*
    /usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so
    /usr/local/lib/python3.11/dist-packages/hopper/*
Proceed (Y/n)? Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.11/dist-package

### Перед запуском нужны веса исходной модели, их необходимо загрузить в формате pth в папку pretrain

Таже, возможно, придется изменить код для загрузки весов модели:
model.load_state_dict(filtered, strict=False)

In [ ]:
!sh /content/RWKV-X/pretrain/training-run_prolong_0.2B.sh

INFO:numexpr.utils:NumExpr defaulting to 12 threads.
INFO:pytorch_lightning.utilities.rank_zero:########## work in progress ##########
2025-12-28 18:02:24.950162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-28 18:02:24.967936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766944944.989870    6732 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766944944.996466    6732 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-